In [1]:
# Dynamic time warping, networkx graph
    # class containing
        # trend id, prices, nearest neighbor, distance to nearest neighbor 
        # if trend has all distance matrices computed, pop to reduce calculations
# dimensionality reduction using pca, t-sne
# data generation, cnn

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy
import time

from dtaidistance import dtw

from database_credentials import get_database_url
from load_transform import load_transform_combine, pivot

In [3]:
database_url = get_database_url()
engine = sqlalchemy.create_engine(database_url)
connection = engine.connect()

In [4]:
sell_data = load_transform_combine(4, connection)
sell_data = pivot(sell_data)
trends = sell_data.to_numpy()

In [34]:
num_trends = trends.shape[0]
distances = np.zeros((num_trends, num_trends))

for x, y in np.ndindex(distances.shape):
    if (x > y):
        distance = dtw.distance(trends[x], trends[y])
        distances[x][y] = distance
        
        # mirror along diagonal to reduce computation time
        distances[y][x] = distance 
        
# fill diagonal so the most similar sequence is not itself
np.fill_diagonal(distances, np.inf) 

In [ ]:
# row index, distance